# Logistic Regression

In [4]:
# Importing libraries for building the neural network
import pandas as pd
import io
import numpy as np
from sklearn.model_selection import train_test_split,StratifiedKFold,cross_val_score,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [5]:
data = pd.read_csv("../Dataset/credit_card_default_train.csv").dropna()
test = pd.read_csv("../Dataset/credit_card_default_test.csv").dropna()
print(data.shape)
print(test.shape)
print(list(data.columns))
print(list(test.columns))
data.head(n=3)

(24000, 25)
(6000, 24)
['Client_ID', 'Balance_Limit_V1', 'Gender', 'EDUCATION_STATUS', 'MARITAL_STATUS', 'AGE', 'PAY_JULY', 'PAY_AUG', 'PAY_SEP', 'PAY_OCT', 'PAY_NOV', 'PAY_DEC', 'DUE_AMT_JULY', 'DUE_AMT_AUG', 'DUE_AMT_SEP', 'DUE_AMT_OCT', 'DUE_AMT_NOV', 'DUE_AMT_DEC', 'PAID_AMT_JULY', 'PAID_AMT_AUG', 'PAID_AMT_SEP', 'PAID_AMT_OCT', 'PAID_AMT_NOV', 'PAID_AMT_DEC', 'NEXT_MONTH_DEFAULT']
['Client_ID', 'Balance_Limit_V1', 'Gender', 'EDUCATION_STATUS', 'MARITAL_STATUS', 'AGE', 'PAY_JULY', 'PAY_AUG', 'PAY_SEP', 'PAY_OCT', 'PAY_NOV', 'PAY_DEC', 'DUE_AMT_JULY', 'DUE_AMT_AUG', 'DUE_AMT_SEP', 'DUE_AMT_OCT', 'DUE_AMT_NOV', 'DUE_AMT_DEC', 'PAID_AMT_JULY', 'PAID_AMT_AUG', 'PAID_AMT_SEP', 'PAID_AMT_OCT', 'PAID_AMT_NOV', 'PAID_AMT_DEC']


Client_ID Balance_Limit_V1 Gender EDUCATION_STATUS MARITAL_STATUS  \
0        A2               1M      M         Graduate          Other   
1        A3               1M      F      High School          Other   
2        A4             100K      F      High School         Single   

            AGE  PAY_JULY  PAY_AUG  PAY_SEP  PAY_OCT         ...          \
0         31-45        -1       -1       -1       -1         ...           
1  Less than 30         0       -1       -1       -1         ...           
2         31-45         4        3        2        2         ...           

   DUE_AMT_OCT  DUE_AMT_NOV  DUE_AMT_DEC  PAID_AMT_JULY  PAID_AMT_AUG  \
0        39418       162772       -13982           3437          6004   
1        43530        80811       124590         151818         46200   
2            0            0            0              0             0   

   PAID_AMT_SEP  PAID_AMT_OCT  PAID_AMT_NOV  PAID_AMT_DEC  NEXT_MONTH_DEFAULT  
0         39418        162772             0        538165                   0  
1         43530         80811           942         33666                   0  
2             0             0             0             0                   1  

[3 rows x 25 columns]

In [6]:
# prediction_var = ['Gender', 'EDUCATION_STATUS', 'MARITAL_STATUS', 
#            'PAY_JULY', 'PAY_AUG', 'PAY_SEP', 'PAY_OCT', 'PAY_NOV', 'PAY_DEC',
#            'DUE_AMT_JULY', 'DUE_AMT_AUG', 'DUE_AMT_SEP','DUE_AMT_OCT','DUE_AMT_NOV','DUE_AMT_DEC', 'PAID_AMT_JULY', 'PAID_AMT_AUG', 'PAID_AMT_SEP', 'PAID_AMT_OCT', 'PAID_AMT_NOV', 'PAID_AMT_DEC']
prediction_var = ['Balance_Limit_V1', 'Gender', 'EDUCATION_STATUS', 'MARITAL_STATUS', 'AGE', 'PAY_JULY', 'PAY_AUG', 'PAY_SEP', 'PAY_OCT', 'PAY_NOV', 'PAY_DEC', 'DUE_AMT_JULY', 'DUE_AMT_AUG', 'DUE_AMT_SEP','DUE_AMT_OCT','DUE_AMT_NOV','DUE_AMT_DEC', 'PAID_AMT_JULY', 'PAID_AMT_AUG', 'PAID_AMT_SEP', 'PAID_AMT_OCT', 'PAID_AMT_NOV', 'PAID_AMT_DEC']
X = data[prediction_var]
Y = data.NEXT_MONTH_DEFAULT
test_id= test['Client_ID']
test = test[prediction_var]

In [7]:
# clean the data
encoder = LabelEncoder()
encoder.fit(X['Balance_Limit_V1'])
(X['Balance_Limit_V1']) = encoder.transform(X['Balance_Limit_V1'])
encoder.fit(X['Gender'])
(X['Gender']) = encoder.transform(X['Gender'])
encoder.fit(X['EDUCATION_STATUS'])
(X['EDUCATION_STATUS']) = encoder.transform(X['EDUCATION_STATUS'])
encoder.fit(X['MARITAL_STATUS'])
(X['MARITAL_STATUS']) = encoder.transform(X['MARITAL_STATUS'])
encoder.fit(X['AGE'])
(X['AGE']) = encoder.transform(X['AGE'])

encoder.fit(test['Balance_Limit_V1'])
(test['Balance_Limit_V1']) = encoder.transform(test['Balance_Limit_V1'])
encoder.fit(test['Gender'])
(test['Gender']) = encoder.transform(test['Gender'])
encoder.fit(test['EDUCATION_STATUS'])
(test['EDUCATION_STATUS']) = encoder.transform(test['EDUCATION_STATUS'])
encoder.fit(test['MARITAL_STATUS'])
(test['MARITAL_STATUS']) = encoder.transform(test['MARITAL_STATUS'])
encoder.fit(test['AGE'])
(test['AGE']) = encoder.transform(test['AGE'])


encoder.fit(Y)
encoded_Y = encoder.transform(Y)

F:\Program Files\anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
F:\Program Files\anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
F:\Program Files\anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.

In [8]:
# converted all the columns to float
X=X.astype(float)
test=test.astype(float)
Y=Y.astype(float)

In [9]:
# standardized the data
sc = StandardScaler()
X = sc.fit_transform(X)
test = sc.fit_transform(test)


## Feature Engineering

In [10]:
# oversampling as the data set is unbalanced
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
os = SMOTE(random_state=0)
os_data_X,os_data_y=os.fit_sample(X, Y)
os_data_X = pd.DataFrame(data=X,columns=prediction_var )
os_data_y= pd.DataFrame(data=Y,columns=['NEXT_MONTH_DEFAULT'])

ModuleNotFoundError: No module named 'imblearn'

In [11]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
rfe = RFE(logreg, 20)
rfe = rfe.fit(os_data_X,os_data_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

[ True  True  True  True False  True  True  True  True  True  True  True
 False  True  True False  True  True  True  True  True  True  True]
[1 1 1 1 4 1 1 1 1 1 1 1 2 1 1 3 1 1 1 1 1 1 1]


In [12]:
# Select only the columns that are useful
prediction_var_corrected = ['Balance_Limit_V1', 'Gender', 'EDUCATION_STATUS', 'MARITAL_STATUS',  'PAY_JULY', 'PAY_AUG', 'PAY_SEP', 'PAY_OCT', 'PAY_NOV', 'PAY_DEC', 'DUE_AMT_JULY', 'DUE_AMT_SEP','DUE_AMT_OCT','DUE_AMT_DEC', 'PAID_AMT_JULY', 'PAID_AMT_AUG', 'PAID_AMT_SEP', 'PAID_AMT_OCT', 'PAID_AMT_NOV', 'PAID_AMT_DEC']
os_data_X = os_data_X[prediction_var_corrected]
import statsmodels.api as sm
logit_model=sm.Logit(os_data_y,os_data_X)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.647467
         Iterations 6
                          Results: Logit
Model:              Logit              Pseudo R-squared: -0.223    
Dependent Variable: NEXT_MONTH_DEFAULT AIC:              31118.4040
Date:               2020-02-15 05:13   BIC:              31280.1201
No. Observations:   24000              Log-Likelihood:   -15539.   
Df Model:           19                 LL-Null:          -12709.   
Df Residuals:       23980              LLR p-value:      1.0000    
Converged:          1.0000             Scale:            1.0000    
No. Iterations:     6.0000                                         
-------------------------------------------------------------------
                     Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
-------------------------------------------------------------------
Balance_Limit_V1    -0.0382   0.0137 -2.7866 0.0053 -0.0650 -0.0113
Gender              -0.0433   0.0136 -3.1

In [13]:
# Refine further
prediction_var_selected = ['Balance_Limit_V1', 'Gender',  'MARITAL_STATUS',  'PAY_JULY', 'PAY_AUG', 'DUE_AMT_JULY', 'PAID_AMT_JULY', 'PAID_AMT_AUG']
os_data_X = os_data_X[prediction_var_selected]

Accuracy of logistic regression classifier on test set: 0.81


## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(os_data_X, Y, test_size=0.3, random_state=0)
logreg = LogisticRegression(C=0.3)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

In [ ]:
test_X = pd.DataFrame(data=test,columns=prediction_var )
test_X = test_X[prediction_var_selected]
y_pred = logreg.predict(test_X)

In [ ]:
# predicting the test values and downloading the csv file
# Y_test= (rfc.predict(test)>0.5)
ID = pd.DataFrame(test_id, columns = ['Client_ID']) 
REV = pd.DataFrame(rfc_predict, columns = ['NEXT_MONTH_DEFAULT']) 
COLS = [ID,REV]
result = pd.concat(COLS, axis=1)
result.columns
result.to_csv('Randresult.csv' , index=False) 

     Client_ID  NEXT_MONTH_DEFAULT
0       A20170                 0.0
1       A16887                 0.0
2        A3430                 0.0
3        A3696                 0.0
4       A20474                 0.0
...        ...                 ...
5995    A11104                 0.0
5996     A2781                 0.0
5997     A1767                 0.0
5998     A1850                 0.0
5999    A26905                 0.0

[6000 rows x 2 columns]


In [ ]:
from google.colab import files

In [ ]:

from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop
model = tf.keras.Sequential([keras.layers.Dense(1, input_shape=[23],activation='sigmoid')])
model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50)

Train on 19200 samples
Epoch 1/50
19200/19200 [==============================] - 1s 46us/sample - loss: 0.6691 - acc: 0.6446
Epoch 2/50
19200/19200 [==============================] - 1s 36us/sample - loss: 0.5159 - acc: 0.7964
Epoch 3/50
19200/19200 [==============================] - 1s 36us/sample - loss: 0.4781 - acc: 0.8081
Epoch 4/50
19200/19200 [==============================] - 1s 38us/sample - loss: 0.4714 - acc: 0.8090
Epoch 5/50
19200/19200 [==============================] - 1s 34us/sample - loss: 0.4700 - acc: 0.8090
Epoch 6/50
19200/19200 [==============================] - 1s 36us/sample - loss: 0.4692 - acc: 0.8098
Epoch 7/50
19200/19200 [==============================] - 1s 38us/sample - loss: 0.4691 - acc: 0.8095
Epoch 8/50
19200/19200 [==============================] - 1s 36us/sample - loss: 0.4689 - acc: 0.8110
Epoch 9/50
19200/19200 [==============================] - 1s 34us/sample - loss: 0.4689 - acc: 0.8093
Epoch 10/50
19200/19200 [==============================] - 

In [ ]:
a=model.predict(X_test)
np.savetxt("foo.csv", a, delimiter=",")
from google.colab import files
files.download("foo.csv")

In [ ]:
print(test[1])

[ 0.61710934 -1.24714324  0.95984434  1.09480937 -0.93266074  0.02382324
  0.12562092  0.16521834  0.20455451  0.25154926  0.26314842 -0.17464607
 -0.13848075 -0.09166864 -0.02773625  0.04983331  0.09537452 -0.25167895
 -0.24260584 -0.19876039 -0.17300372 -0.19031842 -0.180657  ]


In [ ]:
# random forest model creation
from sklearn.model_selection import train_test_split,StratifiedKFold,cross_val_score,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
rfc = RandomForestClassifier(n_estimators=600,  max_features='sqrt',max_depth= 380)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
rfc.fit(X_train,y_train)

# predictions
rfc_predict = rfc.predict(X_test)


In [39]:
# predictions
from sklearn.metrics import accuracy_score
rfc_predict = rfc.predict(X_test)
accuracy_score(y_test, rfc_predict)
# print("=== Confusion Matrix ===")
# print(confusion_matrix(y_test, rfc_predict))
# print(confusion_matrix(y_test, y_pred))

0.82125